# 市场共享

## 目标和前提条件

在这个例子中,我们将向您展示如何解决目标规划问题,该问题涉及将零售商分配给公司的两个部门,以优化几个市场共享目标的权衡。您将学习如何使用 Gurobi Python API 创建问题的混合整数线性规划模型,并使用 Gurobi Optimizer 找到问题的最优解。

这个模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版第 267-268 页和 322-324 页的示例 13。

这个建模示例属于初级水平,我们假设您了解 Python 并且对构建数学优化模型有一定了解。读者还应查阅 Gurobi Python API 的[文档](https://www.gurobi.com/resources/?category-filter=documentation)。

**下载仓库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的仓库。

## 问题描述

一家大公司有两个部门:D1 和 D2。该公司向零售商供应石油和汽油。目标是将每个零售商分配给 D1 部门或 D2 部门。被分配的部门将成为该零售商的供应商。在可能的范围内,分配必须使 D1 控制 40% 的市场,D2 控制剩余的 60%。下表中列出了零售商 M1 到 M23。每个零售商都有石油和汽油的预估市场。零售商 M1 到 M8 在第 1 区,零售商 M9 到 M18 在第 2 区,零售商 M19 到 M23 在第 3 区。某些零售商被认为具有良好的增长前景,归类为 A 组,其他零售商归入 B 组。每个零售商都有一定数量的配送点。
![retailers](retailers.PNG)

我们希望在以下几个类别中实现 D1 和 D2 之间 40%/60% 的划分:
1. 配送点总数
2. 汽油市场控制
3. 第 1 区石油市场控制
4. 第 2 区石油市场控制
5. 第 3 区石油市场控制
6. A 组零售商数量
7. B 组零售商数量。

市场份额可以有 ±5% 的灵活度。即,份额可以在 35%/65% 和 45%/55% 之间变化。目标是最小化与 40%/60% 划分的百分比偏差之和。

## 模型构建

### 集合与索引

$r \in \text{Retailers}=\{\ 1,2,...,23\}$

### 参数

$\text{deliveryPoints}_{r} \in \mathbb{N}^+$: 零售商 $r$ 的配送点数。

$\text{spiritMarket}_{r} \in \mathbb{R}^+$: 零售商 $r$ 的汽油市场(百万加仑)。

$\text{oilMarket1}_{r} \in \mathbb{R}^+$: 零售商 $r$ 在第 1 区的石油市场(百万加仑)。

$\text{oilMarket2}_{r} \in \mathbb{R}^+$: 零售商 $r$ 在第 2 区的石油市场(百万加仑)。

$\text{oilMarket3}_{r} \in \mathbb{R}^+$: 零售商 $r$ 在第 3 区的石油市场(百万加仑)。

$\text{retailerA}_{r} \in \{0,1\}$: 如果零售商 $r$ 属于 A 组,该参数值为 1。

$\text{retailerB}_{r} \in \{0,1\}$: 如果零售商 $r$ 属于 B 组,该参数值为 1。

$\text{deliveryPoints40} \in \mathbb{R}^+$: 配送点总数的 40%。

$\text{deliveryPoints5} \in \mathbb{R}^+$: 配送点总数的 5%。

$\text{spiritMarket40} \in \mathbb{R}^+$: 汽油市场的 40%。

$\text{spiritMarket5} \in \mathbb{R}^+$: 汽油市场的 5%。

$\text{oilMarket1_40} \in \mathbb{R}^+$: 第 1 区石油市场的 40%。

$\text{oilMarket1_5} \in \mathbb{R}^+$: 第 1 区石油市场的 5%。

$\text{oilMarket2_40} \in \mathbb{R}^+$: 第 2 区石油市场的 40%。

$\text{oilMarket2_5} \in \mathbb{R}^+$: 第 2 区石油市场的 5%。

$\text{oilMarket3_40} \in \mathbb{R}^+$: 第 3 区石油市场的 40%。

$\text{oilMarket3_5} \in \mathbb{R}^+$: 第 3 区石油市场的 5%。

$\text{retailerA40} \in \mathbb{R}^+$: A 组零售商数量的 40%。

$\text{retailerA5} \in \mathbb{R}^+$: A 组零售商数量的 5%。

$\text{retailerB40} \in \mathbb{R}^+$: B 组零售商数量的 40%。

$\text{retailerB5} \in \mathbb{R}^+$: B 组零售商数量的 5%。

### 决策变量

$\text{allocate}_{r} \in \{0,1\}$: 这个二元变量在零售商 r 分配给 D1 部门时等于 1,分配给 D2 部门时等于 0。

$\text{deliveryPointsPos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足配送点 40% 目标上的正偏差。

$\text{deliveryPointsNeg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足配送点 40% 目标上的负偏差。

$\text{spiritMarketPos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足汽油市场 40% 目标上的正偏差。

$\text{spiritMarketNeg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足汽油市场 40% 目标上的负偏差。

$\text{oilMarket1Pos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足第 1 区石油市场 40% 目标上的正偏差。

$\text{oilMarket1Neg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足第 1 区石油市场 40% 目标上的负偏差。

$\text{oilMarket2Pos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足第 2 区石油市场 40% 目标上的正偏差。

$\text{oilMarket2Neg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足第 2 区石油市场 40% 目标上的负偏差。

$\text{oilMarket3Pos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足第 3 区石油市场 40% 目标上的正偏差。

$\text{oilMarket3Neg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足第 3 区石油市场 40% 目标上的负偏差。

$\text{retailerAPos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足 A 组零售商数量 40% 目标上的正偏差。

$\text{retailerANeg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足 A 组零售商数量 40% 目标上的负偏差。

$\text{retailerBPos} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足 B 组零售商数量 40% 目标上的正偏差。

$\text{retailerBNeg} \in \mathbb{R}^+$: 这个决策变量衡量零售商分配在满足 B 组零售商数量 40% 目标上的负偏差。

### 约束条件

**配送点**: D1 部门的零售商分配尽可能满足 40% 的配送点目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{deliveryPoints}_{r}*{\text{allocate}_{r}} + \text{deliveryPointsPos} - \text{deliveryPointsNeg}  = \text{deliveryPoints40}
\end{equation}

**汽油市场**: D1 部门的零售商分配尽可能满足 40% 的汽油市场目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{spiritMarket}_{r}*{\text{allocate}_{r}} + \text{spiritMarketPos} - 
\text{spiritMarketNeg}  = \text{spiritMarket40}
\end{equation}

**第 1 区石油市场**: 该区域内 D1 部门的零售商分配尽可能满足 40% 的石油市场目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{oilMarket1}_{r}*{\text{allocate}_{r}} + \text{oilMarket1Pos} - 
\text{oilMarket1Neg}  = \text{oilMarket1_40}
\end{equation}

**第 2 区石油市场**: 该区域内 D1 部门的零售商分配尽可能满足 40% 的石油市场目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{oilMarket2}_{r}*{\text{allocate}_{r}} + \text{oilMarket2Pos} - 
\text{oilMarket2Neg}  = \text{oilMarket2_40}
\end{equation}

**第 3 区石油市场**: 该区域内 D1 部门的零售商分配尽可能满足 40% 的石油市场目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{oilMarket3}_{r}*{\text{allocate}_{r}} + \text{oilMarket3Pos} - 
\text{oilMarket3Neg}  = \text{oilMarket3_40}
\end{equation}

**A 组**: D1 部门的零售商分配尽可能满足 40% 的 A 组零售商数量目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{retailerA40}_{r}*{\text{allocate}_{r}} + \text{retailerAPos} - 
\text{retailerANeg}  = \text{retailerA40}
\end{equation}

**B 组**: D1 部门的零售商分配尽可能满足 40% 的 B 组零售商数量目标。

\begin{equation}
\sum_{r \in \text{Retailers}} \text{retailerB40}_{r}*{\text{allocate}_{r}} + \text{retailerBPos} - 
\text{retailerBNeg}  = \text{retailerB40}
\end{equation}

**灵活度**: 任何市场份额可以有 ±5% 的变化。

$$
\text{deliveryPointsPos} \leq \text{deliveryPoints5}
$$

$$
\text{deliveryPointsNeg}  \leq \text{deliveryPoints5}
$$

$$
\text{spiritMarketPos} \leq \text{spiritMarket5}
$$

$$
\text{spiritMarketNeg}  \leq \text{spiritMarket5}
$$

$$
\text{oilMarket1Pos} \leq \text{oilMarket1_5}
$$

$$
\text{oilMarket1Neg} \leq \text{oilMarket1_5}
$$

$$
\text{oilMarket2Pos} \leq \text{oilMarket2_5}
$$

$$
\text{oilMarket2Neg} \leq \text{oilMarket2_5}
$$

$$
\text{oilMarket3Pos} \leq \text{oilMarket3_5}
$$

$$
\text{oilMarket3Neg} \leq \text{oilMarket3_5}
$$

$$
\text{retailerAPos} \leq \text{retailerA5}
$$

$$
\text{retailerANeg} \leq \text{retailerA5}
$$

$$
\text{retailerBPos} \leq \text{retailerB5}
$$

$$
\text{retailerBNeg} \leq \text{retailerB5}
$$

### 目标函数

**最小化偏差**: 最小化正负偏差之和。

\begin{equation}
\text{最小化} \quad  \text{deliveryPointsPos} + \text{deliveryPointsNeg} + \text{spiritMarketPos} + \text{spiritMarketNeg} +
\text{oilMarket1Pos} + \text{oilMarket1Neg}
\end{equation}

$$
+ \text{oilMarket2Pos} + \text{oilMarket2Neg} + \text{oilMarket3Pos} + \text{oilMarket3Neg} 
$$

$$
+ \text{retailerAPos} + \text{retailerANeg} + \text{retailerBPos} + \text{retailerBNeg}
$$

## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# 使用 Python 3.11 和 Gurobi 11.0 测试

## 输入数据

我们定义模型的所有输入数据。

In [ ]:
# 创建字典来存储配送点和汽油市场(单位:百万加仑)数据

retailers, deliveryPoints, spiritMarket = gp.multidict({
    (1): [11,34],
    (2): [47,411],
    (3): [44,82],
    (4): [25,157],
    (5): [10,5],
    (6): [26,183],
    (7): [26,14],
    (8): [54,215],
    (9): [18,102],
    (10): [51,21],
    (11): [20,54],
    (12): [105,0],
    (13): [7,6],
    (14): [16,96],
    (15): [34,118],
    (16): [100,112],
    (17): [50,535],
    (18): [21,8],
    (19): [11,53],
    (20): [19,28],
    (21): [14,69],
    (22): [10,65],
    (23): [11,27]
})

# 创建字典来存储第1区的石油市场(单位:百万加仑)数据

retailers1,  oilMarket1 = gp.multidict({
    (1): 9,
    (2): 13,
    (3): 14,
    (4): 17,
    (5): 18,
    (6): 19,
    (7): 23,
    (8): 21
})

# 创建字典来存储第2区的石油市场(单位:百万加仑)数据

retailers2,  oilMarket2 = gp.multidict({
    (9): 9,
    (10): 11,
    (11): 17,
    (12): 18,
    (13): 18,
    (14): 17,
    (15): 22,
    (16): 24,
    (17): 36,
    (18): 43
})

# 创建字典来存储第3区的石油市场(单位:百万加仑)数据

retailers3,  oilMarket3 = gp.multidict({
    (19): 6,
    (20): 15,
    (21): 15,
    (22): 25,
    (23): 39
})

# 创建字典来存储A组零售商

groupA,  retailerA = gp.multidict({
    (1): 1,
    (2): 1,
    (3): 1,
    (5): 1,
    (6): 1,
    (10): 1,
    (15): 1,
    (20): 1
})

# 创建字典来存储B组零售商

groupB,  retailerB = gp.multidict({
    (4): 1,
    (7): 1,
    (8): 1,
    (9): 1,
    (11): 1,
    (12): 1,
    (13): 1,
    (14): 1,
    (16): 1,
    (17): 1,
    (18): 1,
    (19): 1,
    (21): 1,
    (22): 1,
    (23): 1
})

# 每个目标的40%和5%值

deliveryPoints40 = 292
deliveryPoints5 = 36.5
spiritMarket40 = 958
spiritMarket5 = 119.75
oilMarket1_40 = 53.6
oilMarket1_5 = 6.7
oilMarket2_40 = 86
oilMarket2_5 = 10.75
oilMarket3_40 = 40
oilMarket3_5 = 5
retailerA40 = 3.2
retailerA5 = 0.4
retailerB40 = 6
retailerB5 = 0.75

## 模型部署

我们创建模型和变量。主要的决策变量是一个二元变量,当零售商分配给 D1 部门时等于 1,分配给 D2 部门时等于 0。其余的决策变量衡量七个 40%/60% 分配目标的正负偏差。

In [ ]:
model = gp.Model('MarketSharing')

# 零售商分配给第1部门的变量
allocate = model.addVars(retailers, vtype=GRB.BINARY, name="allocate")

# 配送点目标的正负偏差

deliveryPointsPos = model.addVar(ub= deliveryPoints5, name='deliveryPointsPos')
deliveryPointsNeg = model.addVar(ub= deliveryPoints5, name='deliveryPointsNeg')

# 汽油市场目标的正负偏差

spiritMarketPos = model.addVar(ub=spiritMarket5, name='spiritMarketPos')
spiritMarketNeg = model.addVar(ub=spiritMarket5, name='spiritMarketNeg')

# 第1区石油市场目标的正负偏差

oilMarket1Pos = model.addVar(ub=oilMarket1_5, name='oilMarket1Pos')
oilMarket1Neg = model.addVar(ub=oilMarket1_5, name='oilMarket1Neg')

# 第2区石油市场目标的正负偏差

oilMarket2Pos = model.addVar(ub=oilMarket2_5, name='oilMarket2Pos')
oilMarket2Neg = model.addVar(ub=oilMarket2_5, name='oilMarket2Neg')

# 第3区石油市场目标的正负偏差

oilMarket3Pos = model.addVar(ub=oilMarket3_5, name='oilMarket3Pos')
oilMarket3Neg = model.addVar(ub=oilMarket3_5, name='oilMarket3Neg')

# A组零售商数量目标的正负偏差

retailerAPos  = model.addVar(ub=retailerA5, name='retailerAPos')
retailerANeg  = model.addVar(ub=retailerA5, name='retailerANeg')

# B组零售商数量目标的正负偏差

retailerBPos  = model.addVar(ub=retailerB5, name='retailerBPos')
retailerBNeg  = model.addVar(ub=retailerB5, name='retailerBNeg')

Using license file c:\gurobi\gurobi.lic


D1 部门的零售商分配尽可能满足 40% 的配送点目标。

In [ ]:
# 配送点约束

DPConstr = model.addConstr((gp.quicksum(deliveryPoints[r]*allocate[r] for r in retailers) 
                            + deliveryPointsPos - deliveryPointsNeg == deliveryPoints40), name='DPConstrs')

D1 部门的零售商分配尽可能满足 40% 的汽油市场目标。

In [ ]:
# 汽油市场约束

SMConstr = model.addConstr((gp.quicksum(spiritMarket[r]*allocate[r] for r in retailers) 
                            + spiritMarketPos - spiritMarketNeg == spiritMarket40), name='SMConstr')

第 1 区内 D1 部门的零售商分配尽可能满足 40% 的石油市场目标。

In [ ]:
# 第1区石油市场约束

OM1Constr = model.addConstr((gp.quicksum(oilMarket1[r]*allocate[r] for r in retailers1) 
                            + oilMarket1Pos - oilMarket1Neg == oilMarket1_40), name='OM1Constr')

第 2 区内 D1 部门的零售商分配尽可能满足 40% 的石油市场目标。

In [ ]:
# 第2区石油市场约束

OM2Constr = model.addConstr((gp.quicksum(oilMarket2[r]*allocate[r] for r in retailers2) 
                            + oilMarket2Pos - oilMarket2Neg == oilMarket2_40), name='OM2Constr')

第 3 区内 D1 部门的零售商分配尽可能满足 40% 的石油市场目标。

In [ ]:
# 第3区石油市场约束

OM3Constr = model.addConstr((gp.quicksum(oilMarket3[r]*allocate[r] for r in retailers3) 
                            + oilMarket3Pos - oilMarket3Neg == oilMarket3_40), name='OM3Constr')

D1 部门的零售商分配尽可能满足 40% 的 A 组零售商数量目标。

In [ ]:
# A组约束

AConstr = model.addConstr((gp.quicksum(retailerA[r]*allocate[r] for r in groupA) 
                            + retailerAPos - retailerANeg == retailerA40), name='AConstr')

D1 部门的零售商分配尽可能满足 40% 的 B 组零售商数量目标。

In [ ]:
# B组约束

BConstr = model.addConstr((gp.quicksum(retailerB[r]*allocate[r] for r in groupB) 
                            + retailerBPos - retailerBNeg == retailerB40), name='BConstr')

最小化正负偏差之和。

In [ ]:
# 目标函数

obj = deliveryPointsPos + deliveryPointsNeg+ spiritMarketPos + spiritMarketNeg + oilMarket1Pos + oilMarket1Neg + oilMarket2Pos + oilMarket2Neg + oilMarket3Pos + oilMarket3Neg + retailerAPos + retailerANeg + retailerBPos + retailerBNeg 

model.setObjective(obj)

In [ ]:
# 验证模型公式

model.write('marketSharing.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 37 columns and 105 nonzeros
Model fingerprint: 0xa5aab3a9
Variable types: 14 continuous, 23 integer (23 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-01, 1e+02]
  RHS range        [3e+00, 1e+03]
Presolve time: 0.00s
Presolved: 7 rows, 37 columns, 105 nonzeros
Variable types: 14 continuous, 23 integer (23 binary)

Root relaxation: objective 0.000000e+00, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    7          -    0.00000      -     -    0s
H    0     0                     131.6000000    0.00000   100%     -    0s
H    0     0                     111.6000000    0.00000   100%     -    0s
H

## 分析

以下是使目标偏差之和最小的零售商分配方案。此外,我们还展示了每个目标如何在 35%/45% 范围内。

In [ ]:
# 输出报告

print("\n\n_________________________________________________________________________________")
print(f"分配给第1部门的最优零售商方案是:")
print("_________________________________________________________________________________")
for r in retailers:
    if(allocate[r].x > 0.5):
        print(f"零售商{r}")

#print(f"\n目标函数的最优值是 {model.objVal}")



_________________________________________________________________________________
The optimal allocation of retailers to Division 1 is:
_________________________________________________________________________________
Retailer2
Retailer6
Retailer7
Retailer9
Retailer12
Retailer13
Retailer14
Retailer15
Retailer23


以下报告验证目标都在可接受的 35% 和 45% 范围内得到满足。

In [ ]:
# 测试解决方案是否在可接受范围内

DeliveryPointsGoal = sum([deliveryPoints[r] for r in retailers if allocate[r].x > 0.5])
spiritMarketGoal = sum([spiritMarket[r] for r in retailers if allocate[r].x > 0.5])
oilMarket1Goal = sum([oilMarket1[r] for r in retailers1 if allocate[r].x > 0.5])
oilMarket2Goal = sum([oilMarket2[r] for r in retailers2 if allocate[r].x > 0.5])
oilMarket3Goal = sum([oilMarket3[r] for r in retailers3 if allocate[r].x > 0.5])
retailerAGoal = sum([retailerA[r] for r in groupA if allocate[r].x > 0.5])
retailerBGoal = sum([retailerB[r] for r in groupB if allocate[r].x > 0.5])

goal_ranges = pd.DataFrame({
    "目标": ["配送点", "汽油市场", "第1区石油市场", "第2区石油市场", "第3区石油市场", "A组", "B组"],
    "最小值_35": [round(val*(0.35/0.40),2) for val in (deliveryPoints40, spiritMarket40, oilMarket1_40, oilMarket2_40, oilMarket3_40, retailerA40, retailerB40)],
    "实际值": [round(val,2) for val in (DeliveryPointsGoal, spiritMarketGoal, oilMarket1Goal, oilMarket2Goal, oilMarket3Goal, retailerAGoal, retailerBGoal)],
    "最大值_45": [round(val*(0.45/0.40),2) for val in (deliveryPoints40, spiritMarket40, oilMarket1_40, oilMarket2_40, oilMarket3_40, retailerA40, retailerB40)],
})
goal_ranges.index=[''] * len(goal_ranges)
goal_ranges

,Goal,Min_35,Actual,Max_45
,Delivery points,255.50,290.0,328.50
,Spirit market,838.25,957.0,1077.75
,Oil market1,46.90,55.0,60.30
,Oil market2,75.25,84.0,96.75
,Oil market3,35.00,39.0,45.00
,Group A,2.80,3.0,3.60
,Group B,5.25,6.0,6.75


## 参考文献

H. Paul Williams, 《数学规划中的模型构建》,第五版。

版权所有 © 2020 Gurobi Optimization, LLC